## Séparer notre ensemble de Question-Réponses
Dans le but d'entrainer plusieurs fois notre modèle sur plusieurs ensemble d'entrainement possible, nous avons créé ce notebook (section) qui explique nos motivations et la façon dont nous avons obtenu tous nos ensembles d'entrainement et de tests. 

Les résultats de chaque expérience/extraction de sets seront sauvegardés dans le fichier ```../datasets/ready-to-train```

In [4]:
# imports
import numpy as np
import pandas as pd

# Constantes
PATH = "../datasets/Nos datasets"
FILE_NAME = "/Dataset_prefinal.tsv"
SEP = "\t"

# lecture et affichage
df_to_split = pd.read_csv(PATH+FILE_NAME, sep=SEP)
print("Nous avons :", len(df_to_split), "couples Q&A.")
df_to_split.head(5)

Nous avons : 655 couples Q&A.


,Question,Reponse
0,Quels sont tes centres d'interets,Je suis interessé par beaucoup de choses. Nous...
1,Quels sont tes centres d'interets,Je suis interessé par une grande variete de su...
2,Quels sont tes sujets preferes,"Mes sujets preferes incluent la robotique, l'i..."
3,Quel est ton numero,Je n'ai pas de numero.
4,Quel est ton numero,"5446, c'est mon numero maintenant."


### Sans conditions sur les sous-ensembles dont se compose le dataset
Notre ensemble étant conçus de plusieurs dialogues, Q&A... d'autres dataset (traduit ou non ; détails dans le fichier ```TODO: À METTRE PLUS TARD```), le but ici est de créer un test set avec K couples de Q&A tirés de façon totalement aléatoire. 

Le test set sera unique quand aux questions, aucune vérifications sur les réponses. Autrement dit, *nous n'auraons pas des questions identiques mais un doublon de réponse pourrait apparaître.*

In [ ]:
from sklearn.model_selection import train_test_split


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e5c503ea-724e-4975-85d2-1122520b6aaa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>